<a href="https://colab.research.google.com/github/Ignacio246/colab/blob/main/clase_18_11_2022_dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%pip install bokeh>=2.4.2

In [1]:
import time
import pandas as pd
import dask.dataframe as dd

In [2]:
df = pd.read_csv('calificaciones_promedio_v2.csv')

In [3]:
df

,Matricula,Nombre,Grupo,Materia 1,Materia 2,Materia 3,Materia 4,Materia 5,Estado,Promedio
0,11111111,Alumno 1,TI11,1,8,1,7,2,regular,3.80000
1,11111112,Alumno 2,TI11,10,7,1,8,5,regular,6.96000
2,11111113,Alumno 3,TI11,4,1,1,3,0,regular,3.19200
3,11111114,Alumno 4,TI11,5,4,3,2,7,regular,4.83840
4,11111115,Alumno 5,TI11,9,10,3,9,2,regular,7.56768


In [4]:
%%time
df.head()

CPU times: user 189 µs, sys: 0 ns, total: 189 µs
Wall time: 196 µs


,Matricula,Nombre,Grupo,Materia 1,Materia 2,Materia 3,Materia 4,Materia 5,Estado,Promedio
0,11111111,Alumno 1,TI11,1,8,1,7,2,regular,3.80000
1,11111112,Alumno 2,TI11,10,7,1,8,5,regular,6.96000
2,11111113,Alumno 3,TI11,4,1,1,3,0,regular,3.19200
3,11111114,Alumno 4,TI11,5,4,3,2,7,regular,4.83840
4,11111115,Alumno 5,TI11,9,10,3,9,2,regular,7.56768


In [5]:
df.shape

(5, 10)

In [7]:
df.describe()

,Matricula,Materia 1,Materia 2,Materia 3,Materia 4,Materia 5,Promedio
count,5.000000e+00,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,1.111111e+07,5.800000,6.000000,1.800000,5.800000,3.200000,5.271616
std,1.581139e+00,3.701351,3.535534,1.095445,3.114482,2.774887,1.923583
min,1.111111e+07,1.000000,1.000000,1.000000,2.000000,0.000000,3.192000
25%,1.111111e+07,4.000000,4.000000,1.000000,3.000000,2.000000,3.800000
50%,1.111111e+07,5.000000,7.000000,1.000000,7.000000,2.000000,4.838400
75%,1.111111e+07,9.000000,8.000000,3.000000,8.000000,5.000000,6.960000
max,1.111112e+07,10.000000,10.000000,3.000000,9.000000,7.000000,7.567680


In [ ]:
from concurrent.futures import process
from dask.distributed import Client

client = Client(n_workers=1, threads_per_worker=4, processes=True, memory_limit='1GB')
client

In [13]:
ddf = dd.read_csv('calificaciones_promedio_v2.csv')

In [14]:
%%time
ddf

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


,Matricula,Nombre,Grupo,Materia 1,Materia 2,Materia 3,Materia 4,Materia 5,Estado,Promedio
npartitions=1,,,,,,,,,,
,int64,object,object,int64,int64,int64,int64,int64,object,float64
,...,...,...,...,...,...,...,...,...,...


In [16]:
%%time
ddf.compute().shape

CPU times: user 133 ms, sys: 26.3 ms, total: 159 ms
Wall time: 1.23 s


(5, 10)

In [17]:
%%time
ddf.head()

CPU times: user 35.5 ms, sys: 5.64 ms, total: 41.2 ms
Wall time: 100 ms


,Matricula,Nombre,Grupo,Materia 1,Materia 2,Materia 3,Materia 4,Materia 5,Estado,Promedio
0,11111111,Alumno 1,TI11,1,8,1,7,2,regular,3.80000
1,11111112,Alumno 2,TI11,10,7,1,8,5,regular,6.96000
2,11111113,Alumno 3,TI11,4,1,1,3,0,regular,3.19200
3,11111114,Alumno 4,TI11,5,4,3,2,7,regular,4.83840
4,11111115,Alumno 5,TI11,9,10,3,9,2,regular,7.56768


In [19]:
%%time
ddf.groupby("Grupo").mean().compute()

CPU times: user 59.6 ms, sys: 7.3 ms, total: 66.9 ms
Wall time: 118 ms


,Matricula,Materia 1,Materia 2,Materia 3,Materia 4,Materia 5,Promedio
Grupo,,,,,,,
TI11,11111113.0,5.8,6.0,1.8,5.8,3.2,5.271616


In [20]:
%%time
ddf.groupby("Grupo").sum().compute()

CPU times: user 52.8 ms, sys: 5.61 ms, total: 58.4 ms
Wall time: 137 ms


,Matricula,Materia 1,Materia 2,Materia 3,Materia 4,Materia 5,Promedio
Grupo,,,,,,,
TI11,55555565,29,30,9,29,16,26.35808


In [23]:
%%time
ddf.groupby("Grupo").max().compute()

CPU times: user 48.9 ms, sys: 8.91 ms, total: 57.8 ms
Wall time: 112 ms


,Matricula,Nombre,Materia 1,Materia 2,Materia 3,Materia 4,Materia 5,Estado,Promedio
Grupo,,,,,,,,,
TI11,11111115,Alumno 5,10,10,3,9,7,regular,7.56768


In [24]:
ddf.groupby(["Grupo","Estado"]).mean().compute()  ##Agrupar 

,,Matricula,Materia 1,Materia 2,Materia 3,Materia 4,Materia 5,Promedio
Grupo,Estado,,,,,,,
TI11,regular,11111113.0,5.8,6.0,1.8,5.8,3.2,5.271616


In [25]:
ddf.columns

Index(['Matricula', 'Nombre', 'Grupo', 'Materia 1', 'Materia 2', 'Materia 3',
       'Materia 4', 'Materia 5', 'Estado', 'Promedio'],
      dtype='object')